# Solutions: CLI Validation

**Complete solutions with explanations**

---

## Solution 1: Build `odibi list` Command

In [ ]:
import click
import yaml
import json
from typing import Dict, Any, List
import fnmatch

@click.group()
def odibi():
    """Odibi Data Pipeline Framework."""
    pass

@odibi.group()
def list_cmd():
    """List available resources."""
    pass

# Rename to 'list' for CLI
odibi.add_command(list_cmd, name='list')

def load_config(config_path: str) -> Dict[str, Any]:
    """Load and parse config file."""
    with open(config_path) as f:
        return yaml.safe_load(f)

def filter_items(items: Dict[str, Any], pattern: str) -> Dict[str, Any]:
    """Filter items by name pattern."""
    if not pattern:
        return items
    return {k: v for k, v in items.items() if fnmatch.fnmatch(k, pattern)}

def format_output(data: Any, format_type: str) -> str:
    """Format data for output."""
    if format_type == 'json':
        return json.dumps(data, indent=2)
    elif format_type == 'yaml':
        return yaml.dump(data, default_flow_style=False)
    else:  # table
        return data  # Already formatted as string

@list_cmd.command()
@click.argument('config', type=click.Path(exists=True))
@click.option('--format', type=click.Choice(['table', 'json', 'yaml']), default='table')
@click.option('--filter', 'filter_pattern', help='Filter by name pattern')
def connections(config: str, format: str, filter_pattern: str):
    """List all connections."""
    cfg = load_config(config)
    conns = cfg.get('connections', {})
    conns = filter_items(conns, filter_pattern)
    
    if format == 'table':
        click.secho("Connections:", fg='cyan', bold=True)
        if not conns:
            click.echo("  (none)")
        else:
            for name, details in conns.items():
                conn_type = details.get('type', 'unknown')
                click.echo(f"  • {name} ({conn_type})")
                if 'catalog' in details:
                    click.echo(f"    Catalog: {details['catalog']}")
                if 'schema' in details:
                    click.echo(f"    Schema: {details['schema']}")
    else:
        output = format_output(conns, format)
        click.echo(output)

@list_cmd.command()
@click.argument('config', type=click.Path(exists=True))
@click.option('--format', type=click.Choice(['table', 'json', 'yaml']), default='table')
@click.option('--filter', 'filter_pattern', help='Filter by name pattern')
def pipelines(config: str, format: str, filter_pattern: str):
    """List all pipelines."""
    cfg = load_config(config)
    pipes = cfg.get('pipelines', {})
    pipes = filter_items(pipes, filter_pattern)
    
    if format == 'table':
        click.secho("Pipelines:", fg='cyan', bold=True)
        if not pipes:
            click.echo("  (none)")
        else:
            for name, details in pipes.items():
                steps = details.get('steps', [])
                step_count = len(steps)
                click.echo(f"  • {name} ({step_count} steps)")
                
                if 'description' in details:
                    click.echo(f"    {details['description']}")
                
                if 'connection' in details:
                    click.echo(f"    Connection: {details['connection']}")
    else:
        output = format_output(pipes, format)
        click.echo(output)

@list_cmd.command()
@click.argument('config', type=click.Path(exists=True))
@click.option('--format', type=click.Choice(['table', 'json', 'yaml']), default='table')
def schemas(config: str, format: str):
    """List available schemas from connections."""
    cfg = load_config(config)
    conns = cfg.get('connections', {})
    
    schemas_by_conn = {}
    for name, details in conns.items():
        if 'schema' in details:
            schemas_by_conn[name] = details['schema']
        elif 'catalog' in details:
            schemas_by_conn[name] = f"{details['catalog']}.default"
    
    if format == 'table':
        click.secho("Schemas:", fg='cyan', bold=True)
        if not schemas_by_conn:
            click.echo("  (none configured)")
        else:
            for conn, schema in schemas_by_conn.items():
                click.echo(f"  • {conn}: {schema}")
    else:
        output = format_output(schemas_by_conn, format)
        click.echo(output)

# Test
if __name__ == "__main__":
    from click.testing import CliRunner
    
    SAMPLE_CONFIG = """
connections:
  prod_db:
    type: databricks
    catalog: main
  staging_db:
    type: databricks
    catalog: staging
  warehouse:
    type: snowflake
    account: abc123

pipelines:
  sales_etl:
    connection: prod_db
    description: "Extract and transform sales data"
    steps:
      - extract:
          sql: "SELECT * FROM sales"
      - transform:
          code: "df['revenue'] = df['price'] * df['quantity']"
      - load:
          table: "sales_clean"
  
  sales_report:
    connection: prod_db
    steps:
      - extract:
          sql: "SELECT * FROM sales_clean"
      - aggregate:
          groupby: ["region"]
      - transform:
          code: "df['margin'] = df['revenue'] - df['cost']"
      - validate:
          checks: ["no_nulls"]
      - load:
          table: "sales_summary"
  
  customer_analysis:
    connection: warehouse
    steps:
      - extract:
          sql: "SELECT * FROM customers"
      - transform:
          code: "df['lifetime_value'] = df['total_purchases'] * df['avg_order']"
"""
    
    runner = CliRunner()
    
    with runner.isolated_filesystem():
        with open('config.yaml', 'w') as f:
            f.write(SAMPLE_CONFIG)
        
        print("=" * 60)
        print("Test 1: List connections")
        print("=" * 60)
        result = runner.invoke(odibi, ['list', 'connections', 'config.yaml'])
        print(result.output)
        
        print("=" * 60)
        print("Test 2: List pipelines with filter")
        print("=" * 60)
        result = runner.invoke(odibi, ['list', 'pipelines', 'config.yaml', '--filter', 'sales*'])
        print(result.output)
        
        print("=" * 60)
        print("Test 3: List connections as JSON")
        print("=" * 60)
        result = runner.invoke(odibi, ['list', 'connections', 'config.yaml', '--format', 'json'])
        print(result.output)

---

## Solution 2: Add Progress Bars

In [ ]:
import click
import time
import yaml
from typing import List, Dict, Any
from dataclasses import dataclass

@dataclass
class StepResult:
    """Result from executing a step."""
    name: str
    duration: float
    rows_processed: int
    success: bool = True

class Pipeline:
    """Mock pipeline for testing."""
    
    def __init__(self, name: str, steps: List[Dict[str, Any]]):
        self.name = name
        self.steps = steps
    
    def run_step(self, step: Dict[str, Any], step_name: str) -> StepResult:
        """Simulate step execution."""
        import random
        duration = random.uniform(0.3, 1.5)
        time.sleep(duration)
        
        return StepResult(
            name=step_name,
            duration=duration,
            rows_processed=random.randint(1000, 50000)
        )

@click.command()
@click.argument('config', type=click.Path(exists=True))
@click.option('--pipeline', '-p', help='Pipeline name to run')
@click.option('--quiet', '-q', is_flag=True, help='No progress output')
@click.option('--summary', is_flag=True, help='Show summary statistics')
def run(
    config: str,
    pipeline: str,
    quiet: bool,
    summary: bool
) -> None:
    """Execute pipeline with progress tracking."""
    
    # Load config
    with open(config) as f:
        cfg = yaml.safe_load(f)
    
    # Get pipeline
    pipelines = cfg.get('pipelines', {})
    if not pipelines:
        click.secho("No pipelines found", fg='red', err=True)
        raise click.Abort()
    
    # Select pipeline
    if pipeline:
        if pipeline not in pipelines:
            click.secho(f"Pipeline '{pipeline}' not found", fg='red', err=True)
            raise click.Abort()
        pipeline_name = pipeline
    else:
        # Use first pipeline
        pipeline_name = list(pipelines.keys())[0]
    
    pipeline_config = pipelines[pipeline_name]
    steps = pipeline_config.get('steps', [])
    
    if not quiet:
        click.secho(f"\nRunning pipeline: {pipeline_name}", fg='cyan', bold=True)
        click.echo()
    
    # Create pipeline
    pipe = Pipeline(pipeline_name, steps)
    
    # Track statistics
    start_time = time.time()
    results: List[StepResult] = []
    
    # Execute with progress bar
    if quiet:
        # No progress bar
        for i, step in enumerate(steps):
            step_name = list(step.keys())[0]
            result = pipe.run_step(step, step_name)
            results.append(result)
    else:
        # With progress bar
        with click.progressbar(
            steps,
            label='Progress',
            show_eta=True,
            item_show_func=lambda s: list(s.keys())[0] if s else ''
        ) as bar:
            for step in bar:
                step_name = list(step.keys())[0]
                step_start = time.time()
                result = pipe.run_step(step, step_name)
                results.append(result)
                
                # Show step completion
                click.echo(f"  ✓ {step_name} ({result.duration:.2f}s)")
    
    total_time = time.time() - start_time
    
    # Show completion
    if not quiet:
        click.echo()
        click.secho("✅ Pipeline completed successfully", fg='green', bold=True)
    
    # Show summary
    if summary:
        click.echo()
        click.secho("Summary:", fg='cyan', bold=True)
        click.echo(f"  Pipeline: {pipeline_name}")
        click.echo(f"  Total time: {total_time:.2f}s")
        click.echo(f"  Steps completed: {len(results)}")
        
        total_rows = sum(r.rows_processed for r in results)
        click.echo(f"  Rows processed: {total_rows:,}")
        
        click.echo()
        click.secho("  Step timings:", fg='cyan')
        for result in results:
            click.echo(f"    • {result.name}: {result.duration:.2f}s ({result.rows_processed:,} rows)")

# Test
if __name__ == "__main__":
    from click.testing import CliRunner
    
    test_config = """
connections:
  db:
    type: databricks

pipelines:
  test_pipeline:
    steps:
      - extract:
          sql: "SELECT * FROM source"
      - transform:
          code: "df['new_col'] = df['old_col'] * 2"
      - validate:
          checks: ["no_nulls"]
      - load:
          table: "output"
"""
    
    runner = CliRunner()
    
    with runner.isolated_filesystem():
        with open('config.yaml', 'w') as f:
            f.write(test_config)
        
        print("=" * 60)
        print("Test: Run with summary")
        print("=" * 60)
        result = runner.invoke(run, ['config.yaml', '--summary'])
        print(result.output)

---

## Solution 3: Config Linter

In [ ]:
import click
import yaml
import re
from typing import List, Dict, Any, Set
from dataclasses import dataclass

@dataclass
class LintResult:
    severity: str  # 'error', 'warning', 'info'
    location: str
    message: str
    rule: str
    fixable: bool = False

class ConfigLinter:
    """Lints Odibi configuration files."""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.results: List[LintResult] = []
    
    def lint(self) -> List[LintResult]:
        """Run all lint checks."""
        self.check_unused_connections()
        self.check_missing_descriptions()
        self.check_missing_explanations()
        self.check_missing_validation()
        self.check_long_pipelines()
        self.check_hardcoded_values()
        return self.results
    
    def check_unused_connections(self):
        """Check for connections that are never used."""
        connections = set(self.config.get('connections', {}).keys())
        used_connections: Set[str] = set()
        
        # Find used connections
        for pipeline_name, pipeline in self.config.get('pipelines', {}).items():
            if 'connection' in pipeline:
                used_connections.add(pipeline['connection'])
        
        # Report unused
        unused = connections - used_connections
        for conn_name in unused:
            self.results.append(LintResult(
                severity='warning',
                location=f"connections.{conn_name}",
                message=f"Connection '{conn_name}' is defined but never used",
                rule='W001',
                fixable=True
            ))
    
    def check_missing_descriptions(self):
        """Check for pipelines without descriptions."""
        for pipeline_name, pipeline in self.config.get('pipelines', {}).items():
            if 'description' not in pipeline:
                self.results.append(LintResult(
                    severity='info',
                    location=f"pipelines.{pipeline_name}",
                    message=f"Pipeline '{pipeline_name}' missing description",
                    rule='I001'
                ))
    
    def check_missing_explanations(self):
        """Check for steps without explanations."""
        for pipeline_name, pipeline in self.config.get('pipelines', {}).items():
            steps = pipeline.get('steps', [])
            
            for i, step in enumerate(steps):
                step_type = list(step.keys())[0]
                step_config = step[step_type]
                
                if isinstance(step_config, dict):
                    if 'explanation' not in step_config:
                        self.results.append(LintResult(
                            severity='error',
                            location=f"pipelines.{pipeline_name}.steps[{i}].{step_type}",
                            message=f"Step missing explanation",
                            rule='E001'
                        ))
    
    def check_missing_validation(self):
        """Check for pipelines without validation steps."""
        for pipeline_name, pipeline in self.config.get('pipelines', {}).items():
            steps = pipeline.get('steps', [])
            
            has_validation = any('validate' in step for step in steps)
            
            if not has_validation and len(steps) > 2:
                self.results.append(LintResult(
                    severity='warning',
                    location=f"pipelines.{pipeline_name}",
                    message=f"Pipeline has no validation step",
                    rule='W002'
                ))
    
    def check_long_pipelines(self, max_steps: int = 10):
        """Check for overly long pipelines."""
        for pipeline_name, pipeline in self.config.get('pipelines', {}).items():
            steps = pipeline.get('steps', [])
            
            if len(steps) > max_steps:
                self.results.append(LintResult(
                    severity='warning',
                    location=f"pipelines.{pipeline_name}",
                    message=f"Long pipeline ({len(steps)} steps, max recommended: {max_steps})",
                    rule='W003'
                ))
    
    def check_hardcoded_values(self):
        """Check for hardcoded dates, numbers in SQL."""
        date_pattern = r"\d{4}-\d{2}-\d{2}"
        
        for pipeline_name, pipeline in self.config.get('pipelines', {}).items():
            steps = pipeline.get('steps', [])
            
            for i, step in enumerate(steps):
                if 'extract' in step:
                    extract_config = step['extract']
                    if isinstance(extract_config, dict) and 'sql' in extract_config:
                        sql = extract_config['sql']
                        
                        # Check for hardcoded dates
                        if re.search(date_pattern, sql):
                            dates = re.findall(date_pattern, sql)
                            self.results.append(LintResult(
                                severity='warning',
                                location=f"pipelines.{pipeline_name}.steps[{i}].extract.sql",
                                message=f"Hardcoded date in SQL: {dates[0]}",
                                rule='W004'
                            ))

@click.command()
@click.argument('config', type=click.Path(exists=True))
@click.option('--fix', is_flag=True, help='Auto-fix issues where possible')
@click.option('--strict', is_flag=True, help='Treat warnings as errors')
def lint(config: str, fix: bool, strict: bool) -> None:
    """Lint configuration file for quality issues."""
    
    # Load config
    with open(config) as f:
        cfg = yaml.safe_load(f)
    
    # Run linter
    linter = ConfigLinter(cfg)
    results = linter.lint()
    
    if not results:
        click.secho("✅ No issues found", fg='green', bold=True)
        return
    
    # Group by severity
    errors = [r for r in results if r.severity == 'error']
    warnings = [r for r in results if r.severity == 'warning']
    info = [r for r in results if r.severity == 'info']
    
    # Display errors
    if errors:
        click.echo()
        click.secho(f"❌ Errors ({len(errors)}):", fg='red', bold=True)
        for result in errors:
            click.echo(f"  • {result.location}: {result.message} [{result.rule}]")
    
    # Display warnings
    if warnings:
        click.echo()
        click.secho(f"⚠️  Warnings ({len(warnings)}):", fg='yellow', bold=True)
        for result in warnings:
            fixable = " (fixable)" if result.fixable else ""
            click.echo(f"  • {result.location}: {result.message} [{result.rule}]{fixable}")
    
    # Display info
    if info:
        click.echo()
        click.secho(f"ℹ️  Suggestions ({len(info)}):", fg='blue', bold=True)
        for result in info:
            click.echo(f"  • {result.location}: {result.message} [{result.rule}]")
    
    # Auto-fix
    if fix:
        fixable_count = sum(1 for r in results if r.fixable)
        if fixable_count > 0:
            click.echo()
            click.secho(f"🔧 Auto-fixing {fixable_count} issue(s)...", fg='cyan')
            # Implementation would modify config and save
            click.secho("Note: Auto-fix not fully implemented", fg='yellow')
    
    # Exit code
    if errors:
        raise click.Abort()
    
    if strict and warnings:
        click.echo()
        click.secho("Failing due to --strict mode", fg='red')
        raise click.Abort()

# Test
if __name__ == "__main__":
    from click.testing import CliRunner
    
    test_config = """
connections:
  prod_db:
    type: databricks
  old_db:
    type: postgres

pipelines:
  sales_etl:
    connection: prod_db
    steps:
      - extract:
          sql: "SELECT * FROM sales WHERE date = '2024-01-01'"
      - transform:
          code: "df['x'] = 1"
      - load:
          table: "output"
"""
    
    runner = CliRunner()
    
    with runner.isolated_filesystem():
        with open('config.yaml', 'w') as f:
            f.write(test_config)
        
        print("=" * 60)
        print("Test: Lint config")
        print("=" * 60)
        result = runner.invoke(lint, ['config.yaml'])
        print(result.output)

---

## Solution 4: Dry-Run Mode

In [ ]:
import click
import yaml
from typing import Dict, Any, List
from dataclasses import dataclass

@dataclass
class StepPlan:
    """Execution plan for a single step."""
    name: str
    type: str
    operations: List[str]
    estimated_time: float
    estimated_rows: int
    details: Dict[str, Any]

class ExecutionPlanner:
    """Plans pipeline execution without running."""
    
    # Historical averages (in reality, would come from metrics)
    STEP_TIME_ESTIMATES = {
        'extract': 2.0,
        'transform': 1.0,
        'aggregate': 1.5,
        'validate': 0.5,
        'load': 1.2,
    }
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
    
    def plan(self, pipeline_name: str) -> Dict[str, Any]:
        """Generate execution plan."""
        pipeline = self.config['pipelines'][pipeline_name]
        steps = pipeline.get('steps', [])
        
        step_plans = []
        total_time = 0
        
        for i, step in enumerate(steps):
            step_type = list(step.keys())[0]
            step_config = step[step_type]
            
            plan = self._plan_step(step_type, step_config, i)
            step_plans.append(plan)
            total_time += plan.estimated_time
        
        return {
            'pipeline_name': pipeline_name,
            'connection': pipeline.get('connection', 'default'),
            'steps': step_plans,
            'total_time': total_time,
            'data_flow': self.get_data_flow(pipeline_name, steps)
        }
    
    def _plan_step(self, step_type: str, config: Any, index: int) -> StepPlan:
        """Plan a single step."""
        estimated_time = self.estimate_step_time(step_type, config)
        estimated_rows = self.estimate_row_count(step_type, config)
        operations = self.analyze_operations(step_type, config)
        
        details = {}
        if isinstance(config, dict):
            if 'sql' in config:
                details['sql'] = config['sql'][:100] + '...' if len(config['sql']) > 100 else config['sql']
            if 'table' in config:
                details['target'] = config['table']
            if 'checks' in config:
                details['checks'] = config['checks']
        
        return StepPlan(
            name=f"step_{index+1}_{step_type}",
            type=step_type,
            operations=operations,
            estimated_time=estimated_time,
            estimated_rows=estimated_rows,
            details=details
        )
    
    def estimate_step_time(self, step_type: str, config: Any) -> float:
        """Estimate step execution time."""
        base_time = self.STEP_TIME_ESTIMATES.get(step_type, 1.0)
        
        # Adjust based on complexity
        if isinstance(config, dict):
            if 'sql' in config:
                # Longer SQL = more time
                base_time *= (1 + len(config['sql']) / 1000)
            if 'code' in config:
                # Count operations
                lines = config['code'].count('\n') + 1
                base_time *= (1 + lines / 10)
        
        return round(base_time, 1)
    
    def estimate_row_count(self, step_type: str, config: Any) -> int:
        """Estimate number of rows."""
        # In reality, would query metadata or use historical stats
        if step_type == 'extract':
            return 10000
        elif step_type == 'aggregate':
            return 100  # Aggregation reduces rows
        else:
            return 10000  # Assume same as input
    
    def analyze_operations(self, step_type: str, config: Any) -> List[str]:
        """Analyze what operations will be performed."""
        operations = []
        
        if isinstance(config, dict):
            if 'code' in config:
                # Parse code for column operations
                code = config['code']
                if 'df[' in code:
                    import re
                    cols = re.findall(r"df\['([^']+)'\]", code)
                    operations.append(f"{len(cols)} column(s) modified")
            
            if 'groupby' in config:
                operations.append(f"Group by {len(config['groupby'])} column(s)")
            
            if 'checks' in config:
                operations.append(f"{len(config['checks'])} validation check(s)")
        
        if not operations:
            operations.append(f"Standard {step_type} operation")
        
        return operations
    
    def get_data_flow(self, pipeline_name: str, steps: List[Dict]) -> str:
        """Generate data flow diagram."""
        flow = []
        
        for step in steps:
            step_type = list(step.keys())[0]
            flow.append(step_type)
        
        return " → ".join(flow)

@click.command()
@click.argument('config', type=click.Path(exists=True))
@click.option('--pipeline', '-p', help='Pipeline name to run')
@click.option('--dry-run', is_flag=True, help='Show execution plan without running')
def run(config: str, pipeline: str, dry_run: bool) -> None:
    """Execute pipeline (or show plan with --dry-run)."""
    
    # Load config
    with open(config) as f:
        cfg = yaml.safe_load(f)
    
    # Get pipeline
    pipelines = cfg.get('pipelines', {})
    if not pipelines:
        click.secho("No pipelines found", fg='red', err=True)
        raise click.Abort()
    
    # Select pipeline
    if pipeline:
        if pipeline not in pipelines:
            click.secho(f"Pipeline '{pipeline}' not found", fg='red', err=True)
            raise click.Abort()
        pipeline_name = pipeline
    else:
        pipeline_name = list(pipelines.keys())[0]
    
    if dry_run:
        # Show execution plan
        planner = ExecutionPlanner(cfg)
        plan = planner.plan(pipeline_name)
        
        click.echo()
        click.secho("Execution Plan", fg='cyan', bold=True)
        click.secho("=" * 60, fg='cyan')
        click.echo()
        
        click.echo(f"Pipeline: {plan['pipeline_name']}")
        click.echo(f"Connection: {plan['connection']}")
        click.echo()
        
        click.secho("Steps:", fg='cyan')
        for i, step in enumerate(plan['steps'], 1):
            click.echo(f"  {i}. {step.type}")
            
            for op in step.operations:
                click.echo(f"     • {op}")
            
            if step.details:
                for key, value in step.details.items():
                    if key == 'sql':
                        click.echo(f"     SQL: {value}")
                    elif key == 'target':
                        click.echo(f"     Target: {value}")
                    elif key == 'checks':
                        click.echo(f"     Checks: {', '.join(value) if isinstance(value, list) else value}")
            
            click.echo(f"     Estimated rows: ~{step.estimated_rows:,}")
            click.echo(f"     Estimated time: {step.estimated_time}s")
            click.echo()
        
        click.secho(f"Total estimated time: {plan['total_time']:.1f}s", fg='yellow', bold=True)
        click.echo()
        
        click.secho("Data Flow:", fg='cyan')
        click.echo(f"  {plan['data_flow']}")
        click.echo()
        
        click.secho("⚠️  This is a dry run. No data will be modified.", fg='yellow', bold=True)
        click.echo()
    else:
        # Actually run pipeline
        click.secho(f"Running pipeline: {pipeline_name}", fg='green')
        # ... execution code ...

# Test
if __name__ == "__main__":
    from click.testing import CliRunner
    
    test_config = """
connections:
  prod_db:
    type: databricks
    catalog: main

pipelines:
  sales_etl:
    connection: prod_db
    steps:
      - extract:
          sql: "SELECT * FROM sales WHERE date >= CURRENT_DATE - 30"
      - transform:
          code: |
            df['revenue'] = df['price'] * df['quantity']
            df['margin'] = df['revenue'] - df['cost']
            df['margin_pct'] = (df['margin'] / df['revenue']) * 100
      - validate:
          checks:
            - no_nulls: ['revenue', 'margin']
            - range_check: {margin_pct: [0, 100]}
      - load:
          table: "sales_clean"
"""
    
    runner = CliRunner()
    
    with runner.isolated_filesystem():
        with open('config.yaml', 'w') as f:
            f.write(test_config)
        
        print("=" * 60)
        print("Test: Dry run")
        print("=" * 60)
        result = runner.invoke(run, ['config.yaml', '--dry-run'])
        print(result.output)

---

## Key Takeaways

### 1. Click Makes CLIs Easy
- Decorators reduce boilerplate
- Automatic validation and help
- Built-in testing support

### 2. Rich Error Messages Matter
- Context helps debugging
- Suggestions guide users
- Color improves readability

### 3. Validation Prevents Issues
- Catch errors before execution
- Enforce quality standards
- Provide actionable feedback

### 4. Testing Is Critical
- CliRunner enables isolated tests
- Test success and failure cases
- Verify exit codes and output

---